<a href="https://colab.research.google.com/github/sowmyarshetty/NNClass/blob/main/amazoncustomerreview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import pandas as pd
import dask.dataframe as dd
import gdown

In [99]:
# Mount Google Drive (For Colab Users)
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
file_id = '1TiwWTY4OT9UCAxOsZCTb9dCmGp34vnDX'

# Construct the download URL (with confirmation bypass)
url = f"https://drive.google.com/uc?export=download&id={file_id}"

# Download the file
gdown.download(url, "amazonreviews_train.csv", quiet=False)



Downloading...
From (original): https://drive.google.com/uc?export=download&id=1TiwWTY4OT9UCAxOsZCTb9dCmGp34vnDX
From (redirected): https://drive.google.com/uc?export=download&id=1TiwWTY4OT9UCAxOsZCTb9dCmGp34vnDX&confirm=t&uuid=f1ea7d42-f376-467b-b934-c3866bd2c9f7
To: /content/amazonreviews_train.csv
100%|██████████| 1.59G/1.59G [00:36<00:00, 42.9MB/s]


'amazonreviews_train.csv'

In [101]:
import pandas as pd

chunk_size = 100000  # Adjust chunk size as needed
chunks = []  # List to store chunk DataFrames

# Read the CSV in chunks
for chunk in pd.read_csv("amazonreviews_train.csv", chunksize=chunk_size):
    chunks.append(chunk)  # Store each chunk

# Concatenate all chunks into a single DataFrame
df = pd.concat(chunks, axis=0)

# Display the first few rows
print(df.head())


   polarity                                              title  \
0         2                     Stuning even for the non-gamer   
1         2              The best soundtrack ever to anything.   
2         2                                           Amazing!   
3         2                               Excellent Soundtrack   
4         2  Remember, Pull Your Jaw Off The Floor After He...   

                                                text  
0  This sound track was beautiful! It paints the ...  
1  I'm reading a lot of reviews saying that this ...  
2  This soundtrack is my favorite music of all ti...  
3  I truly like this soundtrack and I enjoy video...  
4  If you've played the game, you know how divine...  


In [102]:
print(df.tail())


         polarity                                  title  \
3599995         1                          Don't do it!!   
3599996         1          Looks nice, low functionality   
3599997         1             compact, but hard to clean   
3599998         1                     what is it saying?   
3599999         2  Makes My Blood Run Red-White-And-Blue   

                                                      text  
3599995  The high chair looks great when it first comes...  
3599996  I have used this highchair for 2 kids now and ...  
3599997  We have a small house, and really wanted two o...  
3599998  not sure what this book is supposed to be. It ...  
3599999  I agree that every American should read this b...  


In [103]:
df["polarity"].value_counts()

,count
polarity,
2,1800000
1,1800000


In [104]:
print(df.shape)

(3600000, 3)


In [106]:
df.describe()

,polarity
count,3600000.0
mean,1.5
std,0.5
min,1.0
25%,1.0
50%,1.5
75%,2.0
max,2.0


In [107]:
df.dtypes

,0
polarity,int64
title,object
text,object


In [108]:

# Basic text cleaning- Remove special characters or puncatuation marks
df['cleaned_reviews'] = df['text'].str.replace('[^a-zA-Z\s]', '', regex=True)

In [109]:
df.head(5)

,polarity,title,text,cleaned_reviews
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...,This sound track was beautiful It paints the s...
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...,Im reading a lot of reviews saying that this i...
2,2,Amazing!,This soundtrack is my favorite music of all ti...,This soundtrack is my favorite music of all ti...
3,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...,I truly like this soundtrack and I enjoy video...
4,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine...",If youve played the game you know how divine t...




*   What is the the BERT Tokenizer doing?
*   It converts the cleaned reviews to list because hugging face tokenizers work with lists
* BERT appies word piece tokenization . Converts text into subword tokens and assigns each token a unique id
* padding = True . Ensures that all tokenized sequences in the batch are the same length by adding padding tokens to shorter sequences. This is important because transformers process fixed-length inputs in batches
* truncation = True . Shortens any long reviews that exceed the model's maximum token lime ( e:g 512 for BERT). Ensures that excesskive long reviews dont break the models input size limit
* converts the tokenzied output into a pytorch tensor (torch.tensor) format
* required for training or inference in pytorch-based models
* if using tensorflow , we have to pass th e parameter as return_tensors = "tf"



In [ ]:
from transformers import BertTokenizer

# Initialize a tokenizer from a pretrained model (e.g., BERT)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Tokenize the reviews
tokens = tokenizer(df['cleaned_reviews'].tolist(), padding=True, truncation=True, return_tensors="pt")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
tokens = tokenizer(df['cleaned_reviews'].tolist(), padding=True, truncation=True, return_tensors="pt")

In [4]:
import torch

# Save the tokenized output (dictionary of tensors)
torch.save(tokens, 'tokenized_data.pt')

NameError: name 'tokens' is not defined



*   Do not remove stop words from the tokens if we are using the dataset to do
sentiment analysis (classifying reviews as positive / negative or neutral)

*  Do not remove stop words from the tokens if we are classifying reviews into various categories (electronics, clothing etc) and may be summarization of the review



In [ ]:
# Display tokenized output (first 2 examples)
print(tokens.input_ids[:2])
print(tokens.attention_mask[:2])

NameError: name 'tokens' is not defined